# SIRVD-DP: A COVID-19 prediction model of deep learning based on time-dependant SIRVD

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
google_dir =  "/content/drive/MyDrive/"

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from scipy.optimize import leastsq 
import scipy as sp   
import math
import os
import re
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np 

%matplotlib inline
plt.rcParams['font.sans-serif']=['SimHei'] 
plt.rcParams['axes.unicode_minus']=False 
uk = 'United Kingdom'
ch = 'China'
it = 'Italy'
fr = 'France'
sp = 'Spain'
ge = 'Germany'
ko = 'Korea, South'
br = 'Brazil'
india = 'India'
us = 'US'

country_num = {ch: 1439323774, india: 1380004385, us: 331002647, uk: 67886004, it: 60461828, fr: 67564251, sp: 46754783, ge: 83783945,
               ko: 51269183, br: 212559409}
google_dir =  "/content/drive/MyDrive/"
modelSavePath = google_dir + "model/"
figSavePath = google_dir + "figure/"
confirmDataPath = google_dir + "confirmedGlobal.csv"
deathDataPath = google_dir + "deathGlobal.csv"
recoveredDataPath = google_dir + "recoveredGlobal.csv"
confirmUSDataPath = google_dir + "confirmedUS.csv"
deathUSDataPath = google_dir + "deathUS.csv"
recoveredUSDataPath = google_dir + "recoveredUS.csv"
# download
import requests
confirmedGlobalUrl = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
deathGlobalUrl = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
recoveredGlobalUrl = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv"
confirmedUSUrl = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv"
deathUSUrl = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv"
recoveredUSUrl = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_US.csv"
vaccinationUrl = "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/vaccinations.csv"
vaccinationDataPath = google_dir + "vaccinations.csv"

def downloadCsv(url, fileName):    
    r = requests.get(url)
    with open(fileName,'wb') as f:
        f.write(r.content)
        print("download completed")
downloadCsv(confirmedGlobalUrl, confirmDataPath)
downloadCsv(deathGlobalUrl, deathDataPath)
downloadCsv(recoveredGlobalUrl, recoveredDataPath)
downloadCsv(confirmedUSUrl, confirmUSDataPath)
downloadCsv(deathUSUrl, deathUSDataPath)
downloadCsv(recoveredUSUrl, recoveredUSDataPath)
downloadCsv(vaccinationUrl, vaccinationDataPath)

In [ ]:
# read data
def readDeathDataByCountry(countryName):
    death_data = pd.read_csv(deathDataPath)
    data = death_data[(death_data['Country/Region'] == countryName)].iloc[:,4:]
    # & (death_data['Province/State'].isna())
    return data.sum()
def readConfirmedDataByCountry(countryName):
    recover_data = pd.read_csv(confirmDataPath)
    data = recover_data[(recover_data['Country/Region'] == countryName)].iloc[:,4:]
    return data.sum()
def readRecoveredDataByCountry(countryName):
    recover_data = pd.read_csv(recoveredDataPath)
    data = recover_data[(recover_data['Country/Region'] == countryName)].iloc[:,4:]
    return data.sum()
def load_IR(name, N, t,T=None):
    if name == 'sars_bj':
        sars_bj = pd.read_excel('sars_bj.xlsx')
        R = sars_bj['R']
        D = sars_bj['D']
        C = sars_bj['C']
        if T == None:
          T = len(R)-1
        R = R.tolist()
        D = D.tolist()
        C = C.tolist()
    else:
        R = readRecoveredDataByCountry(name)
        D = readDeathDataByCountry(name)
        C = readConfirmedDataByCountry(name) 
        if T == None:
          T = len(R)-1
        R = R.tolist()[t:T+1]
        D = D.tolist()[t:T+1]
        C = C.tolist()[t:T+1]
    R = np.array(R)
    D = np.array(D)
    C = np.array(C)
    I = C - R - D
    datestart=datetime.datetime.strptime("2020-1-22",'%Y-%m-%d')
    datestart += datetime.timedelta(days=+t)
    dateend = datetime.datetime.strptime("2020-1-22",'%Y-%m-%d') 
    dateend += datetime.timedelta(days=T)
    print("load " + name + " IRDC from ", datestart.strftime('%Y-%m-%d'), " to ", dateend.strftime('%Y-%m-%d'), ", total: ", len(I))
    return I,R,D,C
def create_assist_date(datestart = None,dateend = None):
    if datestart is None:
        datestart = '2016-01-01'
    if dateend is None:
        dateend = datetime.datetime.now().strftime('%Y-%m-%d')
    datestart=datetime.datetime.strptime(datestart,'%Y-%m-%d')
    dateend=datetime.datetime.strptime(dateend,'%Y-%m-%d')
    date_list = []
    date_list.append(datestart.strftime('%Y-%m-%d'))
    while datestart<dateend:
        datestart+=datetime.timedelta(days=+1)
        date_list.append(datestart.strftime('%Y-%m-%d'))
    return date_list

def predictPlot(I, I_t):
    date_X = create_assist_date("2020-1-22", "2021-10-01")
    X = np.arange(0, len(I_t))
    ax  = plt.figure(figsize=(13, 8))
    sns.lineplot(X[:len(I_t)],I_t,label="Predict Infected")
    sns.lineplot(X[:len(I)], I, label = 'Current Infected')
    plt.xlabel('Date')
    plt.ylabel('Number of active infections')
    plt.title('SIR Model')
    
def plot(data,label=None):
    X = np.arange(0, len(data))
    ax  = plt.figure(figsize=(13, 8))
    sns.lineplot(X[:len(data)],data,label=label)

def readVaccinationDataByCountry(countryName):
  vaccinationData = pd.read_csv(vaccinationDataPath)
  vacData = vaccinationData[vaccinationData['location'] == countryName]
  datestart = datetime.datetime.strptime(vacData['date'].iloc[0],'%Y-%m-%d')
  dateend = datetime.datetime.strptime(vacData['date'].iloc[-1],'%Y-%m-%d')
  startDate = datetime.datetime.strptime("2020-1-22",'%Y-%m-%d')
  t = (datestart - startDate).days
  T = (dateend - startDate).days
  print("load vaccinatin data: ", countryName, ", start: ", datestart, ", end: ", dateend, ", total: ", (dateend - datestart).days+1, ", t: ", t, " T: ", T)
  V = vacData['people_vaccinated'].tolist()
  V = np.array(V)
  for i in range(0, len(V)):
    if np.isnan(V[i]):
      V[i] = V[i-1]
  return V,t,T
india_V,t,T = readVaccinationDataByCountry("India")
ch_data = load_IR(ch, country_num[ch], 0)
us_data = load_IR(us, country_num[us], 0)
india_data = load_IR(india, country_num[india], 0)

load vaccinatin data:  India , start:  2021-01-15 00:00:00 , end:  2021-05-27 00:00:00 , total:  133 , t:  359  T:  491
load China IRDC from  2020-01-22  to  2021-06-01 , total:  497
load US IRDC from  2020-01-22  to  2021-06-01 , total:  497
load India IRDC from  2020-01-22  to  2021-06-01 , total:  497


In [ ]:
def IRDCV(country):
  V,t,T = readVaccinationDataByCountry(country)
  I,R,D,C = load_IR(country, country_num[country], t, T)
  N = country_num[country]
  S = N - I - R - D - V
  #ax = plt.figure(figsize=(20, 15))
  #plt.plot(I, label='I')
  #plt.plot(R, label='R')
  #plt.plot(D, label='D')
  #plt.plot(C, label='C')
  #plt.plot(V, label='V')
  #plt.legend(loc='best')

  # S->V: alpha
  # I->D: delta
  # I->R: gamma
  # S->I: beta
  # R->S: sigma = 0
  alpha = np.zeros(len(I)-1)
  delta = np.zeros(len(I)-1)
  gamma = np.zeros(len(I)-1)
  beta = np.zeros(len(I)-1)
  sigma = np.zeros(len(I)-1)
  for i in range(0, len(I)-1):
    alpha[i] = (V[i+1] - V[i]) / S[i]
    delta[i] = (D[i+1] - D[i]) / I[i]
    gamma[i] = (R[i+1] - R[i]) / I[i]
    beta[i] = (I[i+1] - I[i] + R[i+1] - R[i] + D[i+1] - D[i]) * N / (I[i] * S[i])
    sigma[i] = (I[i+1] - I[i] + R[i+1] - R[i] + D[i+1] - D[i] + S[i+1] - S[i] + V[i+1] - V[i]) / R[i]
  return S,I,R,V,D,C,alpha, delta, gamma, beta, sigma
  
def normalization(data):
  new_data = data.astype('float32')
  max_value = np.max(new_data)
  min_value = np.min(new_data)
  scalar = max_value - min_value
  new_data = list(map(lambda x: np.array((x) / scalar), new_data))
  return np.array(new_data), scalar
S,I,R,V,D,C,alpha, delta, gamma, beta, sigma = IRDCV(india)

load vaccinatin data:  India , start:  2021-01-15 00:00:00 , end:  2021-05-27 00:00:00 , total:  133 , t:  359  T:  491
load India IRDC from  2021-01-15  to  2021-05-27 , total:  133


In [ ]:

S,I,R,V,D,alpha, delta, gamma, beta, sigma = IRDCV(india)
S, scalar_S = normalization(S)
I, scalar_I = normalization(I)
R, scalar_R = normalization(R)
V, scalar_V = normalization(V)
D, scalar_D = normalization(D)
alpha, scalar_alpha = normalization(alpha)
beta, scalar_beta = normalization(beta)
gamma, scalar_gamma = normalization(gamma)
delta, scalar_delta = normalization(delta)
T = len(beta)
window = 3
n = 1
days = 1
dataX, dataY = [], []
print("dataset length: ", T, " window size: ", window, " prediction days: ", days, " feature num: ", n)
for i in range(T - window - days):
  matrix = []
  # matrix.append(S[i:(i+window)])
  # matrix.append(I[i:(i+window)])
  # matrix.append(R[i:(i+window)])
  # matrix.append(V[i:(i+window)])
  # matrix.append(D[i:(i+window)])
  matrix.append(beta[i:(i+window)])
  # matrix.append(gamma[i:(i+window)])
  # matrix.append(alpha[i:(i+window)])
  # matrix.append(delta[i:(i+window)])
  n = len(matrix)
  dataX.append(matrix)
  dataY.append(beta[(i + window):(i+window+days)])
dataX = np.array(dataX)
dataY = np.array(dataY)

load vaccinatin data:  India , start:  2021-01-15 00:00:00 , end:  2021-05-27 00:00:00 , total:  133 , t:  359  T:  491
load India IRDC from  2021-01-15  to  2021-05-27 , total:  133
数据集总长度:  132  窗口大小:  3  预测天数:  1  特征数量:  1


In [ ]:
def createDataset(data, window=2, days=1):
  dataset = data
  dataset = dataset.astype('float32')
  max_value = np.max(dataset)
  min_value = np.min(dataset)
  scalar = max_value - min_value
  dataset = list(map(lambda x: np.array(x / scalar), dataset))
  dataX, dataY = [], []
  for i in range(len(dataset) - window - days):
      a = dataset[i:(i + window)]
      dataX.append(a)
      a = dataset[(i + window):(i+window+days)]
      dataY.append(a)
  return np.array(dataX), np.array(dataY), scalar
def splitTrainAndTest(data_X, data_Y, ratio=0.7):
  window = data_X.shape[2]
  days = data_Y.shape[1]
  train_size = int(len(data_X) * ratio)
  train_X = data_X[:train_size]
  train_Y = data_Y[:train_size]
  test_X = data_X[train_size:]
  test_Y = data_Y[train_size:]
  return train_X, train_Y, test_X, test_Y, train_size
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
def printScore(y_true, y_pred):
  if y_true.shape != y_pred.shape:
    y_pred = y_pred.reshape(-1,1)
  print(f"(MSE)：{mean_squared_error(y_true, y_pred)}")
  print(f"(RMSE)：{np.sqrt(mean_squared_error(y_true, y_pred))}")
  print(f"(NRMSE)：{np.sqrt(mean_squared_error(y_true, y_pred))/y_true.mean()}")
  print(f"R^2：{r2_score(y_true, y_pred)}")
  print(f"(MAPE)：{mean_absolute_percentage_error(y_true, y_pred)}")

def computeSmape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

def getScore(y_true, y_pred):
  if y_true.shape != y_pred.shape:
    y_pred = y_pred.reshape(-1,1)
  mse = mean_squared_error(y_true, y_pred)
  rmse = np.sqrt(mean_squared_error(y_true, y_pred))
  nrmse = np.sqrt(mean_squared_error(y_true, y_pred))/y_true.mean()
  r2 = r2_score(y_true, y_pred)
  mape = mean_absolute_percentage_error(y_true, y_pred)
  return (mse, rmse,nrmse, r2, mape)

def plotComparison(real, predict):
  if real.shape != predict.shape:
    predict = predict.reshape(-1,1)
  ax = plt.figure(figsize=(20, 15))
  plt.plot(predict, 'r', label='prediction')
  plt.plot(real, 'b', label='real')
  plt.legend(loc='best')
def plotComparisonScatter(real, predict):
  if real.shape != predict.shape:
    predict = predict.reshape(-1,1)
  ax = plt.figure(figsize=(20, 15))
  plt.plot(np.arange(0,len(predict),1), predict, 'o',color = 'r', label='prediction')
  plt.plot(np.arange(0,len(real),1), real, 'o','b', label='real')
  plt.legend(loc='best')

In [ ]:
from tensorflow.keras.layers import Dense, Input, Conv1D, Conv2D, GlobalAveragePooling2D, GlobalAveragePooling1D
from tensorflow.keras.initializers import he_normal
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adagrad
from tensorflow import keras
import sys
import os
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from keras.datasets import mnist
from keras.layers import Dense, LSTM, Bidirectional, Conv1D, Conv2D, MaxPooling1D, MaxPool2D, Flatten
from keras.layers.wrappers import Bidirectional
#from keras.utils import to_categorical
from keras.models import Sequential
import tensorflow as tf
from numpy.random import seed
seed(7)

def VanillaLstm(input_dim, window_size, output_dim, optimizer='adam', learning_rate=0.01):
  #parameters for LSTM
  nb_lstm_outputs = 16  
  #build model
  model = Sequential()
  model.add(LSTM(units=nb_lstm_outputs, input_shape=[input_dim, window_size]))
  model.add(Dense(units=output_dim,
                  input_dim=input_dim,
                  activation='linear'))
  # 'sgd', 'rmsprop', 'adam', 'adagrad'
  if optimizer=='adam':
    optimizer = Adam(learning_rate=learning_rate)
  elif optimizer=='sgd':
    optimizer = SGD(learning_rate=learning_rate)
  elif optimizer=='rmsprop':
    optimizer = RMSprop(learning_rate=learning_rate)
  elif optimizer=='adagrad':
    optimizer = Adagrad(learning_rate=learning_rate)
  return model

# Bi LSTM 
def BiDirectionalLstm(input_dim, window_size, output_dim):
    model = Sequential()
    model.add(Bidirectional(LSTM(16, activation ='relu'), input_shape = [input_dim, window_size]))
    model.add(Dense(input_dim))
    model.add(Dense(output_dim))
    return model

# Stacked-LSTM
def StackedLstm(input_dim, window_size, output_dim):
    model = keras.models.Sequential([
        keras.layers.LSTM(16, return_sequences=True, input_shape=[input_dim, window_size], activation='relu',
                          recurrent_activation='sigmoid'),
        keras.layers.LSTM(32, activation='relu', return_sequences=True, recurrent_activation='sigmoid'),
        keras.layers.LSTM(16, activation='relu', recurrent_activation='sigmoid'),
        Dense(output_dim)
    ])
    return model

# GRU
def GRU(input_dim, window_size, output_dim):
    model = keras.models.Sequential([
        keras.layers.GRU(16, return_sequences=True, input_shape=[input_dim, window_size], activation='relu',
                         recurrent_activation='sigmoid', reset_after=True),
        keras.layers.GRU(32, activation='relu', return_sequences=True, recurrent_activation='sigmoid',
                         reset_after=True),
        keras.layers.GRU(16, activation='relu', recurrent_activation='sigmoid', reset_after=True),
        Dense(output_dim)
    ])
    return model
    
def testSingleModel(model, data, window, days, verbose=0):
  data_X, data_Y, scalar = createDataset(data, window, days)
  train_X, train_Y, test_X, test_Y = splitTrainAndTest(data_X, data_Y)
  print("train_X shape: ", train_X.shape)
  print("train_Y shape: ", train_Y.shape)
  model = model(window, days)
  model.compile(loss='mean_squared_error',optimizer='adam')
  model.fit(train_X, train_Y, epochs=530, batch_size=128, verbose=verbose)
  data_X = data_X.reshape(-1, 1, window)
  y_pred = model.predict(data_X)
  y_real = data_Y * scalar
  y_pred = y_pred * scalar
  printScore(y_real, y_pred)
  plotComparison(y_real, y_pred)

class LossHistory(keras.callbacks.Callback):
  def on_train_begin(self, logs={}):
      self.losses = {'batch':[], 'epoch':[]}
      # self.accuracy = {'batch':[], 'epoch':[]}
      # self.val_loss = {'batch':[], 'epoch':[]}
      # self.val_acc = {'batch':[], 'epoch':[]}

  def on_batch_end(self, batch, logs={}):
      self.losses['batch'].append(logs.get('loss'))
      # self.accuracy['batch'].append(logs.get('acc'))
      # self.val_loss['batch'].append(logs.get('val_loss'))
      # self.val_acc['batch'].append(logs.get('val_acc'))

  def on_epoch_end(self, batch, logs={}):
      self.losses['epoch'].append(logs.get('loss'))
      # self.accuracy['epoch'].append(logs.get('acc'))
      # self.val_loss['epoch'].append(logs.get('val_loss'))
      # self.val_acc['epoch'].append(logs.get('val_acc'))

  def loss_plot(self, loss_type):
      iters = range(len(self.losses[loss_type]))
      plt.figure()
      # acc
      #plt.plot(iters, self.accuracy[loss_type], 'r', label='train acc')
      # loss
      plt.plot(iters, self.losses[loss_type], 'g', label='train loss')
      # if loss_type == 'epoch':
      #     # val_acc
      #     plt.plot(iters, self.val_acc[loss_type], 'b', label='val acc')
      #     # val_loss
      #     plt.plot(iters, self.val_loss[loss_type], 'k', label='val loss')
      plt.grid(True)
      plt.xlabel(loss_type)
      plt.ylabel('acc-loss')
      plt.legend(loc="upper right")
      plt.show()

In [ ]:
history = LossHistory()
model = StackedLstm(n, window, days)
model.compile(loss='mean_squared_error',optimizer='adam')
train_X, train_Y, test_X, test_Y, trainSize = splitTrainAndTest(dataX, dataY)
model.fit(train_X, train_Y, epochs=2000, batch_size=128, verbose=1,callbacks=[history])

In [ ]:
history.loss_plot('epoch')
print("train：")
y_pred = model.predict(train_X)
y_real = train_Y * scalar_beta
y_pred = y_pred * scalar_beta
printScore(y_real, y_pred)
plotComparison(y_real, y_pred)
plotComparisonScatter(y_real, y_pred)
print("test")
y_pred = model.predict(test_X)
y_real = test_Y * scalar_beta
y_pred = y_pred * scalar_beta
printScore(y_real, y_pred)
plotComparison(y_real, y_pred)
plotComparisonScatter(y_real, y_pred)

In [ ]:
# SIRVD-DP
S,I,R,V,D,C,alpha, delta, gamma, beta, sigma = IRDCV(india)
S, scalar_S = normalization(S)
I, scalar_I = normalization(I)
R, scalar_R = normalization(R)
V, scalar_V = normalization(V)
D, scalar_D = normalization(D)
def moving_average(xt):
  for i in range(2, len(xt)-1):
    xt[i] = (xt[i-1] + xt[i])/2
  return xt

alpha, scalar_alpha = normalization(alpha)
beta, scalar_beta = normalization(beta)
gamma, scalar_gamma = normalization(gamma)
delta, scalar_delta = normalization(delta)

beta = moving_average(beta)
gamma = moving_average(gamma)
delta = moving_average(beta)

T = len(beta)
window = 3
n = 1
days = 1

def getFeatureXY(feature_str = "I", output_str = "I"):
  dataX, dataY = [], []
  for i in range(T - window - days):
    matrix = []
    if 'S' in feature_str:
      matrix.append(S[i:(i+window)])
    if 'I' in feature_str:
      matrix.append(I[i:(i+window)])
    if 'R' in feature_str:
      matrix.append(R[i:(i+window)])
    if 'V' in feature_str:
      matrix.append(V[i:(i+window)])
    if 'D' in feature_str:
      matrix.append(D[i:(i+window)])
    if 'b' in feature_str:
      matrix.append(beta[i:(i+window)])
    if 'g' in feature_str:
      matrix.append(gamma[i:(i+window)])
    if 'a' in feature_str:
      matrix.append(alpha[i:(i+window)])
    if 'd' in feature_str:
      matrix.append(delta[i:(i+window)])
    dataX.append(matrix)
    #
    if 'S' in output_str:
      dataY.append(S[(i + window):(i+window+days)])
    if 'I' in output_str:
      dataY.append(I[(i + window):(i+window+days)])
    if 'R' in output_str:
      dataY.append(R[(i + window):(i+window+days)])
    if 'V' in output_str:
      dataY.append(V[(i + window):(i+window+days)])
    if 'D' in output_str:
      dataY.append(D[(i + window):(i+window+days)])
    if 'b' in output_str:
      dataY.append(beta[(i + window):(i+window+days)])
    if 'g' in output_str:
      dataY.append(gamma[(i + window):(i+window+days)])
    if 'a' in output_str:
      dataY.append(alpha[(i + window):(i+window+days)])
    if 'd' in output_str:
      dataY.append(delta[(i + window):(i+window+days)])
  n = len(dataX[0])
  print(f" input features: {feature_str} output feature: {output_str} total length: {T} window size: {window} prediction days: {days} feature nums: {n}")
  dataX = np.array(dataX)
  dataY = np.array(dataY)
  return dataX, dataY

dataX_S, dataY_S = getFeatureXY("S","S")
dataX_I, dataY_I = getFeatureXY("I","I")
dataX_R, dataY_R = getFeatureXY("R","R")
dataX_V, dataY_V = getFeatureXY("V","V")
dataX_D, dataY_D = getFeatureXY("D","D")
dataX_beta, dataY_beta = getFeatureXY("b","b")
dataX_gamma, dataY_gamma = getFeatureXY("g","g")
dataX_delta, dataY_delta = getFeatureXY("d","d")
dataX_alpha, dataY_alpha = getFeatureXY("a","a")

In [ ]:
model_set = {'VanillaLstm':VanillaLstm, "BiDirectionalLstm":BiDirectionalLstm, "StackedLstm":StackedLstm, "GRU":GRU}
def getPredictedParam(model, dataX, dataY, scalar, verbose=1):
  history = LossHistory()
  n = dataX[0].shape[0]
  model = model(n, window, days)
  model.compile(loss='mean_squared_error',optimizer='adam')
  train_X, train_Y, test_X, test_Y, start_size = splitTrainAndTest(dataX, dataY)
  train_start_time = datetime.datetime.now()
  print(f"begin {train_start_time}")
  model.fit(train_X, train_Y, epochs=3000, batch_size=128, verbose=0,callbacks=[history])
  total_time = datetime.datetime.now() - train_start_time
  print(f"end {total_time}")
  # history.loss_plot('epoch')
  
  y_pred = model.predict(train_X)
  y_train_real = train_Y * scalar
  y_pred = y_pred * scalar
  train_score = getScore(y_train_real, y_pred)
  if verbose == 1:
    plotComparison(y_train_real, y_pred)
    #plotComparisonScatter(y_real, y_pred)
  y_train_pred = y_pred

  y_pred = model.predict(test_X)
  y_test_real = test_Y * scalar
  y_pred = y_pred * scalar
  test_score = getScore(y_test_real, y_pred)
  if verbose == 1:
    plotComparison(y_test_real, y_pred)
    #plotComparisonScatter(y_real, y_pred)
  y_test_pred = y_pred
  print(f"train: {train_score} test: {test_score}")
  return y_train_pred, y_test_pred, y_train_real, y_test_real, train_score, test_score

def getParamPredictedResult(dataX_param, dataY_param, scalar_param):
  model_result = []
  model_train_score = []
  model_test_score = []
  for i in range(10):
    for model in model_set:
      beta_train_pred, beta_test_pred, beta_train_real, beta_test_real, beta_train_score, beta_test_score = getPredictedParam(model_set[model], dataX_param, dataY_param, scalar_param)
      model_result.append((beta_train_pred, beta_test_pred, beta_train_real, beta_test_real))
      train_score = list(beta_train_score)
      train_score.insert(0, model)
      model_train_score.append(train_score)
      test_score = list(beta_test_score)
      test_score.insert(0, model)
      model_test_score.append(test_score)
  return (model_result, model_train_score, model_test_score)

model_result = []
model_train_score = []
model_test_score = []
for i in range(10):
  for model in model_set:
    beta_train_pred, beta_test_pred, beta_train_real, beta_test_real, beta_train_score, beta_test_score = getPredictedParam(model_set[model], dataX_beta, dataY_beta, scalar_beta)
    model_result.append((beta_train_pred, beta_test_pred, beta_train_real, beta_test_real))
    train_score = list(beta_train_score)
    train_score.insert(0, model)
    model_train_score.append(train_score)
    test_score = list(beta_test_score)
    test_score.insert(0, model)
    model_test_score.append(test_score)
param_model = getParamPredictedResult(dataX_alpha, dataY_alpha, scalar_alpha)

In [ ]:
beta_model = (model_result, model_train_score, model_test_score)
df2 = open(modelSavePath + "delta_model.pickle",'wb')
pickle.dump(param_model,df2)
df2.close()


In [ ]:
import pickle
df_beta = open(modelSavePath + "beta_model.pickle",'rb')
beta_model = pickle.load(df_beta)
df_beta.close()
df_gamma = open(modelSavePath + "gamma_model.pickle",'rb')
gamma_model = pickle.load(df_gamma)
df_gamma.close()
df_delta = open(modelSavePath + "delta_model.pickle",'rb')
delta_model = pickle.load(df_delta)
df_delta.close()
dataY_I_real = dataY_I * scalar_I 
dataY_S_real = dataY_S * scalar_S
dataY_beta_real = dataY_beta * scalar_beta
dataY_gamma_real = dataY_gamma * scalar_gamma 
dataY_delta_real = dataY_delta * scalar_delta 

In [ ]:
SIRDP_train_result = []
SIRDP_test_result = []
for i in range(len(beta_model[0])):
  beta_train_pred, beta_test_pred, beta_train_real, beta_test_real = beta_model[0][i]
  gamma_train_pred, gamma_test_pred, gamma_train_real, gamma_test_real = gamma_model[0][i]
  delta_train_pred, delta_test_pred, delta_train_real, delta_test_real = delta_model[0][i]
  start_size = int(len(dataY_beta_real) * 0.7)
  pred_I = []
  for t in range(start_size, len(dataY_I_real)):
    predI = dataY_I_real[t-1] * ( 1 + beta_test_pred[t-start_size-1] * dataY_S_real[t-1] / country_num[india] - gamma_test_pred[t-start_size-1] - delta_test_pred[t-start_size-1])
    pred_I.append(predI)
  y_real = dataY_I_real[start_size+1:]
  y_pred = np.array(pred_I[1:])
  test_result = getScore(y_real, y_pred)
  SIRDP_test_result.append(test_result)

  train_pred_I = [0]
  for t in range(1, len(beta_train_pred)):
    predI = dataY_I_real[t-1] * ( 1 + beta_train_pred[t-1] * dataY_S_real[t-1] / country_num[india] - gamma_train_pred[t-1] - delta_train_pred[t-1])
    train_pred_I.append(predI)
  y_real = dataY_I_real[1:start_size]
  y_pred = np.array(train_pred_I[1:])
  train_result = getScore(y_real, y_pred)
  SIRDP_train_result.append(train_result)


In [ ]:
df_SIRDP_train = pd.DataFrame(SIRDP_train_result, columns=["mse","rmse","nrmse","r2","mape"])
df_SIRDP_test = pd.DataFrame(SIRDP_test_result, columns=["mse","rmse","nrmse","r2","mape"])
df_SIRDP_train

In [ ]:
beta_train_pred, beta_test_pred, beta_train_real, beta_test_real, beta_train_score, beta_test_score = getPredictedParam(StackedLstm, dataX_beta, dataY_beta, scalar_beta, verbose=1)
gamma_train_pred, gamma_test_pred, gamma_train_real, gamma_test_real, gamma_train_score, gamma_test_score = getPredictedParam(StackedLstm, dataX_gamma, dataY_gamma, scalar_gamma)
delta_train_pred, delta_test_pred, delta_train_real, delta_test_real, delta_train_score, delta_test_score = getPredictedParam(StackedLstm, dataX_delta, dataY_delta, scalar_delta)
dataY_I = dataY_I * scalar_I 
dataY_S = dataY_S * scalar_S
dataY_beta = dataY_beta * scalar_beta
dataY_gamma = dataY_gamma * scalar_gamma 
dataY_delta = dataY_delta * scalar_delta 

In [ ]:
start_size = int(len(dataY_beta) * 0.7)

pred_I = []
for t in range(start_size, len(dataY_I)):
  predI = dataY_I[t-1] * ( 1 + beta_test_pred[t-start_size-1] * dataY_S[t-1] / country_num[india] - gamma_test_pred[t-start_size-1] - delta_test_pred[t-start_size-1])
  pred_I.append(predI)
y_real = dataY_I[start_size+1:]
y_pred = np.array(pred_I[1:])
print("test：")
printScore(y_real, y_pred)
plotComparison(y_real, y_pred)
plotComparisonScatter(y_real, y_pred)

train_pred_I = [0]
for t in range(1, len(beta_train_pred)):
  predI = dataY_I[t-1] * ( 1 + beta_train_pred[t-1] * dataY_S[t-1] / country_num[india] - gamma_train_pred[t-1] - delta_train_pred[t-1])
  train_pred_I.append(predI)
y_real = dataY_I[1:start_size]
y_pred = np.array(train_pred_I[1:])
print("train：")
printScore(y_real, y_pred)
plotComparison(y_real, y_pred)
plotComparisonScatter(y_real, y_pred)
